In [20]:
# Zelle 1: Installation der notwendigen Bibliotheken
!pip install PyGithub ipywidgets
print("✅ Bibliotheken wurden erfolgreich installiert.")


✅ Bibliotheken wurden erfolgreich installiert.


In [21]:
# Zelle 2: Interaktive Eingabefelder
import ipywidgets as widgets
from IPython.display import display

style = {'description_width': 'initial'}
github_user_widget = widgets.Text(description="GitHub-Benutzer/Orga:", value="bireggbaum-beep", style=style)
repo_name_widget = widgets.Text(description="Repository-Name:", value="site", style=style)
target_folder_widget = widgets.Text(description="Neuer Bild-Ordner:", placeholder="z.B. images", style=style)
github_token_widget = widgets.Password(description="GitHub-Token:", style=style)
delete_originals_widget = widgets.Checkbox(value=False, description='Originalbilder nach dem Kopieren löschen', style=style)

print("Bitte überprüfen Sie die Daten und fügen Sie Ihr funktionierendes Token ein.")
display(github_user_widget, repo_name_widget, target_folder_widget, github_token_widget, delete_originals_widget)


Bitte überprüfen Sie die Daten und fügen Sie Ihr funktionierendes Token ein.


Text(value='bireggbaum-beep', description='GitHub-Benutzer/Orga:', style=DescriptionStyle(description_width='i…

Text(value='site', description='Repository-Name:', style=DescriptionStyle(description_width='initial'))

Text(value='', description='Neuer Bild-Ordner:', placeholder='z.B. images', style=DescriptionStyle(description…

Password(description='GitHub-Token:', style=DescriptionStyle(description_width='initial'))

Checkbox(value=False, description='Originalbilder nach dem Kopieren löschen', style=DescriptionStyle(descripti…

In [26]:
# Zelle 3: Datei-Upload
from google.colab import files

print("Bitte laden Sie Ihre 'products.json'-Datei hoch:")
uploaded = files.upload()

if not uploaded:
    print("❌ Es wurde keine Datei hochgeladen. Bitte Zelle erneut ausführen.")
else:
    json_file_name = list(uploaded.keys())[0]
    json_content_bytes = uploaded[json_file_name]
    print(f"✅ Datei '{json_file_name}' erfolgreich hochgeladen.")


Bitte laden Sie Ihre 'products.json'-Datei hoch:


Saving products (7).json to products (7).json
✅ Datei 'products (7).json' erfolgreich hochgeladen.


In [28]:
# Zelle 4 (Final, Rekursive Version)
import base64
import json
from github import Github, Auth
from github.GithubException import UnknownObjectException

# --- NEUE REKURSIVE FUNKTION ---
def find_and_update_images_recursive(data_structure, target_folder, found_images_set):
    """
    Durchläuft eine verschachtelte Python-Struktur (aus JSON), findet alle 'image'-Felder,
    aktualisiert ihre Pfade und sammelt die Bildnamen.
    """
    if isinstance(data_structure, dict):  # Wenn es ein Objekt/Dictionary ist...
        for key, value in data_structure.items():
            if key == "image" and isinstance(value, str) and value:
                # 'image'-Feld gefunden!
                base_image_name = value.split('/')[-1]
                found_images_set.add(base_image_name)  # Zum Set der zu kopierenden Bilder hinzufügen

                # Den Pfad im JSON aktualisieren
                data_structure[key] = f"{target_folder}/{base_image_name}"
            else:
                # Andernfalls weiter in die Tiefe suchen
                find_and_update_images_recursive(value, target_folder, found_images_set)

    elif isinstance(data_structure, list):  # Wenn es eine Liste ist...
        for item in data_structure:
            # Für jedes Element in der Liste die Suche fortsetzen
            find_and_update_images_recursive(item, target_folder, found_images_set)

    # Andere Datentypen (Strings, Zahlen etc.) werden ignoriert.

# --- HAUPTSKRIPT ---
def run_final_recursive_script():
    # --- 1. Daten auslesen ---
    try:
        user = github_user_widget.value
        repo_name = repo_name_widget.value
        target_folder = target_folder_widget.value.strip('/') # Entfernt '/' am Anfang/Ende
        token = github_token_widget.value
        delete_originals = delete_originals_widget.value

        if not all([user, repo_name, target_folder, token]):
            print("❌ Fehler: Bitte füllen Sie alle Felder in Zelle 2 aus.")
            return
        products_data = json.loads(json_content_bytes.decode('utf-8'))
    except NameError:
        print("❌ Fehler: Haben Sie Zelle 3 ausgeführt und eine Datei hochgeladen?")
        return
    except Exception as e:
        print(f"Fehler beim Einlesen der Eingaben: {e}")
        return

    print("▶️ Starte den Prozess...")
    if delete_originals:
        print("⚠️ HINWEIS: Die Option zum Löschen der Originalbilder ist aktiviert.")

    # --- 2. Mit GitHub verbinden ---
    try:
        auth = Auth.Token(token)
        g = Github(auth=auth)
        repo = g.get_repo(f"{user}/{repo_name}")
        print(f"✅ Erfolgreich mit dem Repository '{repo.full_name}' verbunden.")
    except Exception as e:
        print(f"❌ Fehler bei der Verbindung zu GitHub: {e}")
        return

    # --- 3. Bilder mit der rekursiven Funktion finden und JSON anpassen ---
    images_to_copy = set() # Ein Set, um Duplikate automatisch zu vermeiden
    updated_products_data = products_data.copy() # Kopie der Daten für die Bearbeitung

    find_and_update_images_recursive(updated_products_data, target_folder, images_to_copy)

    if not images_to_copy:
        print("ℹ️ Keine Bilder im Feld 'image' in der JSON-Datei gefunden.")
        return
    print(f"🔎 {len(images_to_copy)} einzigartige Bilder zur Verarbeitung in der JSON gefunden.")

    # --- 4. Bilder auf GitHub kopieren ---
    successfully_copied = {} # Speichert {bildname: sha} für die Lösch-Option
    for image_name in images_to_copy:
        try:
            file_content = repo.get_contents(image_name)
            new_path = f"{target_folder}/{image_name}"
            commit_message = f"Kopiere Bild {image_name} nach {target_folder}"

            try:
                existing_file = repo.get_contents(new_path)
                repo.update_file(new_path, commit_message, file_content.decoded_content, existing_file.sha)
                print(f"🔄 Bild '{new_path}' aktualisiert.")
            except UnknownObjectException:
                repo.create_file(new_path, commit_message, file_content.decoded_content)
                print(f"✅ Bild '{new_path}' erstellt.")

            successfully_copied[image_name] = file_content.sha # SHA für späteres Löschen speichern
        except UnknownObjectException:
            print(f"⚠️ Warnung: Bild '{image_name}' wurde nicht im Hauptverzeichnis gefunden und konnte nicht kopiert werden.")
        except Exception as e:
            print(f"❌ Fehler beim Kopieren von '{image_name}': {e}")

    print(f"\n--- Kopiervorgang abgeschlossen: {len(successfully_copied)} Bilder verarbeitet. ---")

    # --- 5. Originalbilder löschen (falls Option aktiviert) ---
    if delete_originals and successfully_copied:
        print("\n--- Beginne mit dem Löschen der Originalbilder... ---")
        deleted_count = 0
        for image_name, sha in successfully_copied.items():
            try:
                repo.delete_file(image_name, f"Lösche Originalbild {image_name}", sha)
                print(f"🗑️ Originalbild '{image_name}' gelöscht.")
                deleted_count += 1
            except Exception as e:
                print(f"❌ Fehler beim Löschen von '{image_name}': {e}")
        print(f"--- Löschvorgang abgeschlossen: {deleted_count} Bilder gelöscht. ---")

    # --- 6. Aktualisierte 'products.json' hochladen ---
    try:
        updated_json_string = json.dumps(updated_products_data, indent=2, ensure_ascii=False)
        commit_message = "Aktualisiere Bildpfade in products.json"
        try:
            json_file = repo.get_contents("products.json")
            repo.update_file("products.json", commit_message, updated_json_string, json_file.sha)
            print(f"\n🔄 'products.json' erfolgreich aktualisiert.")
        except UnknownObjectException:
            repo.create_file("products.json", commit_message, updated_json_string)
            print(f"\n✅ 'products.json' wurde neu erstellt.")
        print("\n🎉 Prozess erfolgreich abgeschlossen!")
    except Exception as e:
        print(f"\n❌ Kritischer Fehler beim Hochladen der 'products.json': {e}")

# Führe die Hauptfunktion aus
run_final_recursive_script()


▶️ Starte den Prozess...
✅ Erfolgreich mit dem Repository 'bireggbaum-beep/site' verbunden.
🔎 25 einzigartige Bilder zur Verarbeitung in der JSON gefunden.
⚠️ Warnung: Bild 'wollschals-dunkelbraun.jpg' wurde nicht im Hauptverzeichnis gefunden und konnte nicht kopiert werden.
⚠️ Warnung: Bild 'bel-lino-rosa.jpg' wurde nicht im Hauptverzeichnis gefunden und konnte nicht kopiert werden.
⚠️ Warnung: Bild 'stulpen-dunkelbraun.jpg' wurde nicht im Hauptverzeichnis gefunden und konnte nicht kopiert werden.
⚠️ Warnung: Bild 'wendemutzen-hellgrau.jpg' wurde nicht im Hauptverzeichnis gefunden und konnte nicht kopiert werden.
⚠️ Warnung: Bild 'merino-star-lagune.jpg' wurde nicht im Hauptverzeichnis gefunden und konnte nicht kopiert werden.
⚠️ Warnung: Bild 'biskaya-neue-farbe-2.jpg' wurde nicht im Hauptverzeichnis gefunden und konnte nicht kopiert werden.
⚠️ Warnung: Bild 'wendemutzen-hellbraun.jpg' wurde nicht im Hauptverzeichnis gefunden und konnte nicht kopiert werden.
⚠️ Warnung: Bild 'wollkue